In [8]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import glob
from pathlib import Path
import os
from omegaconf import OmegaConf
from pprint import pprint as pp
from icecream import ic
from typing import List

project = Path().resolve().parent
configs = Path(project) / "config"
data_paths = OmegaConf.load(configs / "data.yaml")
print(OmegaConf.to_yaml(data_paths))

raw: ../data/raw
interim: ../data/interim
processed: ../data/processed



In [9]:
data_int = Path(data_paths.processed) / "modsec_audit_new.csv"
df = pd.read_csv(data_int, on_bad_lines="warn", engine="python")

In [10]:
df

,http_version,id,layer_type,local_ip,local_port,method,reference,remote_ip,remote_port,req_accept,...,ssrf_value,status_code,target,targeted_filed,timestamp,txn_id,uri,xss,xss_severity,xss_value
0,1.1,tHKgIuCZ,NaN,127.0.0.1,1234,GET,NaN,127.0.0.1,37530,*/*,...,NaN,401,NaN,NaN,08/Jun/2025:07:05:46 +0000,1.749366e+11,/api/v1/checkouts/user,NaN,NaN,NaN
1,1.1,625GoFZO,SINGLE_LAYERED,127.0.0.1,1234,POST,"o1,12o1,2o12,1v14,25t:urlDecodeUnio59,12o59,2o...",127.0.0.1,37562,*/*,...,NaN,200,sql_injection,NaN,08/Jun/2025:07:07:13 +0000,1.749366e+11,/api/v1/auths/login,False,NaN,NaN
2,1.1,u4sde1wQ,SINGLE_LAYERED,127.0.0.1,1234,POST,"o1,12o1,2o12,1v14,25t:urlDecodeUnio59,12o59,2o...",127.0.0.1,37562,*/*,...,NaN,200,sql_injection,NaN,08/Jun/2025:07:07:16 +0000,1.749366e+11,/api/v1/auths/login,False,NaN,NaN
3,1.1,FQylZXhA,SINGLE_LAYERED,127.0.0.1,1234,POST,"o1,12o1,2o12,1v14,13t:urlDecodeUnio59,12o59,2o...",127.0.0.1,37562,*/*,...,NaN,401,sql_injection,NaN,08/Jun/2025:07:07:21 +0000,1.749366e+11,/api/v1/auths/login,False,NaN,NaN
4,1.1,lR82yFF4,SINGLE_LAYERED,127.0.0.1,1234,POST,"o1,12o1,2o12,1v14,16t:urlDecodeUnio59,12o59,2o...",127.0.0.1,37562,*/*,...,NaN,403,sql_injection,NaN,08/Jun/2025:07:07:27 +0000,1.749366e+11,/api/v1/auths/login,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,1.1,VDm69Pj3,SINGLE_LAYERED,127.0.0.1,1234,POST,"o1,12o1,2o12,1v14,26t:urlDecodeUnio59,12o59,2o...",127.0.0.1,45610,*/*,...,NaN,401,sql_injection,NaN,08/Jun/2025:08:09:54 +0000,1.749370e+11,/api/v1/auths/login,False,NaN,NaN
407,1.1,SFrEbHSo,SINGLE_LAYERED,127.0.0.1,1234,POST,"o1,12o1,2o12,1v14,25t:urlDecodeUnio59,12o59,2o...",127.0.0.1,45610,*/*,...,NaN,403,sql_injection,NaN,08/Jun/2025:08:09:57 +0000,1.749370e+11,/api/v1/auths/login,False,NaN,NaN
408,1.1,rHolkoh9,SINGLE_LAYERED,127.0.0.1,1234,POST,"o1,12o1,2o12,1v14,25t:urlDecodeUnio59,12o59,2o...",127.0.0.1,45610,*/*,...,NaN,200,sql_injection,NaN,08/Jun/2025:08:10:01 +0000,1.749370e+11,/api/v1/auths/login,False,NaN,NaN
409,1.1,uDtZKPWn,SINGLE_LAYERED,127.0.0.1,1234,POST,"o1,12o1,2o12,1v14,25t:urlDecodeUnio59,12o59,2o...",127.0.0.1,45610,*/*,...,NaN,200,sql_injection,NaN,08/Jun/2025:08:10:05 +0000,1.749370e+11,/api/v1/auths/login,False,NaN,NaN


In [11]:
# List out important columns to be used
DEFAULT_KEEP_COLS: List[str] = [
    "id",
    "timestamp",
    "layer_type",
    "method",
    "local_port",
    "remote_port",
    "req_cf_ray",
    "req_content_length",
    "resp_content_length",
    "resp_vary",
    "status_code",
    "target"
]

In [12]:
df.columns = df.columns.str.strip()
df = df.rename(columns=str.lower)
df

,http_version,id,layer_type,local_ip,local_port,method,reference,remote_ip,remote_port,req_accept,...,ssrf_value,status_code,target,targeted_filed,timestamp,txn_id,uri,xss,xss_severity,xss_value
0,1.1,tHKgIuCZ,NaN,127.0.0.1,1234,GET,NaN,127.0.0.1,37530,*/*,...,NaN,401,NaN,NaN,08/Jun/2025:07:05:46 +0000,1.749366e+11,/api/v1/checkouts/user,NaN,NaN,NaN
1,1.1,625GoFZO,SINGLE_LAYERED,127.0.0.1,1234,POST,"o1,12o1,2o12,1v14,25t:urlDecodeUnio59,12o59,2o...",127.0.0.1,37562,*/*,...,NaN,200,sql_injection,NaN,08/Jun/2025:07:07:13 +0000,1.749366e+11,/api/v1/auths/login,False,NaN,NaN
2,1.1,u4sde1wQ,SINGLE_LAYERED,127.0.0.1,1234,POST,"o1,12o1,2o12,1v14,25t:urlDecodeUnio59,12o59,2o...",127.0.0.1,37562,*/*,...,NaN,200,sql_injection,NaN,08/Jun/2025:07:07:16 +0000,1.749366e+11,/api/v1/auths/login,False,NaN,NaN
3,1.1,FQylZXhA,SINGLE_LAYERED,127.0.0.1,1234,POST,"o1,12o1,2o12,1v14,13t:urlDecodeUnio59,12o59,2o...",127.0.0.1,37562,*/*,...,NaN,401,sql_injection,NaN,08/Jun/2025:07:07:21 +0000,1.749366e+11,/api/v1/auths/login,False,NaN,NaN
4,1.1,lR82yFF4,SINGLE_LAYERED,127.0.0.1,1234,POST,"o1,12o1,2o12,1v14,16t:urlDecodeUnio59,12o59,2o...",127.0.0.1,37562,*/*,...,NaN,403,sql_injection,NaN,08/Jun/2025:07:07:27 +0000,1.749366e+11,/api/v1/auths/login,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,1.1,VDm69Pj3,SINGLE_LAYERED,127.0.0.1,1234,POST,"o1,12o1,2o12,1v14,26t:urlDecodeUnio59,12o59,2o...",127.0.0.1,45610,*/*,...,NaN,401,sql_injection,NaN,08/Jun/2025:08:09:54 +0000,1.749370e+11,/api/v1/auths/login,False,NaN,NaN
407,1.1,SFrEbHSo,SINGLE_LAYERED,127.0.0.1,1234,POST,"o1,12o1,2o12,1v14,25t:urlDecodeUnio59,12o59,2o...",127.0.0.1,45610,*/*,...,NaN,403,sql_injection,NaN,08/Jun/2025:08:09:57 +0000,1.749370e+11,/api/v1/auths/login,False,NaN,NaN
408,1.1,rHolkoh9,SINGLE_LAYERED,127.0.0.1,1234,POST,"o1,12o1,2o12,1v14,25t:urlDecodeUnio59,12o59,2o...",127.0.0.1,45610,*/*,...,NaN,200,sql_injection,NaN,08/Jun/2025:08:10:01 +0000,1.749370e+11,/api/v1/auths/login,False,NaN,NaN
409,1.1,uDtZKPWn,SINGLE_LAYERED,127.0.0.1,1234,POST,"o1,12o1,2o12,1v14,25t:urlDecodeUnio59,12o59,2o...",127.0.0.1,45610,*/*,...,NaN,200,sql_injection,NaN,08/Jun/2025:08:10:05 +0000,1.749370e+11,/api/v1/auths/login,False,NaN,NaN


In [17]:
keep = list()
cols_to_keep = keep or DEFAULT_KEEP_COLS
cols_in_file = [c for c in cols_to_keep if c in df.columns]
df = df[cols_in_file].copy()
df['target'] = df['target'].fillna('ssrf')
df.to_csv(Path(data_paths.interim) / "modsec_audit_train_v1_new.csv")
df

,id,timestamp,layer_type,method,local_port,remote_port,req_cf_ray,req_content_length,resp_content_length,resp_vary,status_code,target
0,tHKgIuCZ,08/Jun/2025:07:05:46 +0000,NaN,GET,1234,37530,94c68aeecb38409a-SIN,NaN,50,Origin,401,ssrf
1,625GoFZO,08/Jun/2025:07:07:13 +0000,SINGLE_LAYERED,POST,1234,37562,94c68d0f48d28b59-HKG,86.0,33,Origin,200,sql_injection
2,u4sde1wQ,08/Jun/2025:07:07:16 +0000,SINGLE_LAYERED,POST,1234,37562,94c68d2548a98b59-HKG,86.0,33,Origin,200,sql_injection
3,FQylZXhA,08/Jun/2025:07:07:21 +0000,SINGLE_LAYERED,POST,1234,37562,94c68d420d3a8b59-HKG,74.0,62,Origin,401,sql_injection
4,lR82yFF4,08/Jun/2025:07:07:27 +0000,SINGLE_LAYERED,POST,1234,37562,94c68d65be578b59-HKG,77.0,64,Origin,403,sql_injection
...,...,...,...,...,...,...,...,...,...,...,...,...
406,VDm69Pj3,08/Jun/2025:08:09:54 +0000,SINGLE_LAYERED,POST,1234,45610,94c6e8e07c808b81-HKG,87.0,62,Origin,401,sql_injection
407,SFrEbHSo,08/Jun/2025:08:09:57 +0000,SINGLE_LAYERED,POST,1234,45610,94c6e8f46f648b81-HKG,86.0,64,Origin,403,sql_injection
408,rHolkoh9,08/Jun/2025:08:10:01 +0000,SINGLE_LAYERED,POST,1234,45610,94c6e90ef8b08b81-HKG,86.0,33,Origin,200,sql_injection
409,uDtZKPWn,08/Jun/2025:08:10:05 +0000,SINGLE_LAYERED,POST,1234,45610,94c6e928da2a8b81-HKG,86.0,33,Origin,200,sql_injection


In [18]:
from collections import Counter
Counter(df['target'])

Counter({'xss': 179, 'BENIGN': 125, 'ssrf': 67, 'sql_injection': 40})